In [1]:
import pandas as pd
import numpy as np
import re
import toolz.itertoolz as itz
import tqdm

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [2]:
fpath = r"C:\Users\student\Downloads\MetaMorpheusVignette\MetaMorpheusVignette\2024-07-12-12-55-48\Task2-SearchTask\AllPSMs.psmtsv"
df = pd.read_csv(fpath, sep='\t')

C:\Users\student\AppData\Local\Temp\ipykernel_24100\144969561.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fpath, sep='\t')


In [3]:
#df.head(3)

In [4]:
#df.columns, df.size

In [5]:
def get_offset(startend): return int(re.search(r'\d+', startend).group())

def get_mod_locs(seq, offset=0):
    if 'Cu[I]' in seq:
        seq = seq.replace('Cu[I]', 'Cu(I)')        
        #print(parts)
    mods = []
    for peptide in seq.split('|'):
        peptide_offset = offset
        parts = re.split("\[|\]", str(peptide))
        try: 
            if '' == parts[0]:
                #mods+=[[parts[2][0]+str(offset), parts[1]]]
                mods+=[[str(peptide_offset), parts[1]]]
                parts=parts[2:]
            for l1, l2 in itz.partition(2, parts):
                #mods+=[[l1[-1] + str(offset+len(l1)-1), l2]]
                mods+=[[str(peptide_offset+len(l1)-1), l2]]
                peptide_offset+=len(l1)
        except:
            print('Error Here:', seq, parts)
        
    if mods: return mods
    else: return [[None, None]]


#row = 6983
#offset = get_offset(df.at[row, 'Start and End Residues In Protein'])
#list(get_mod_locs(df.at[row, 'Full Sequence'], offset=offset)), offset
"""
if 'localized_mods' not in df.columns:
    mods = [get_mod_locs(i, offset=get_offset(j)) for i, j in df[['Full Sequence', 'Start and End Residues In Protein']].values]
    df.insert(17, 'localized_mods', mods)
"""
#df

"\nif 'localized_mods' not in df.columns:\n    mods = [get_mod_locs(i, offset=get_offset(j)) for i, j in df[['Full Sequence', 'Start and End Residues In Protein']].values]\n    df.insert(17, 'localized_mods', mods)\n"

In [6]:
df2 = df[(df['Protein Accession']=='P55095') & (df['QValue']<=0.01)]

In [7]:
def get_range_values(range_str):
    return [int(i) for i in re.findall('\d+', range_str)]

In [13]:
aa_occ = {}
ranges = {}


#already assuming protein is the same and QValue<=0.01
for index, row in df2.iterrows():
    offset = get_offset(row['Start and End Residues In Protein'])
    mods = list(get_mod_locs(row['Full Sequence'], offset=offset))
    range = row['Start and End Residues In Protein']
    
    if range not in ranges:
        ranges[range] = {'count':0, 
                         'values':get_range_values(range)}
    ranges[range]['count']+=row['Full Sequence'].count('|')+1
    
    
    for mod in mods:
        if mod[0]:
            if mod[0] not in aa_occ:
                aa_occ[mod[0]] = {'mods':{mod[1]:0}, 'total':0}
            elif mod[1] not in aa_occ[mod[0]]['mods']:
                aa_occ[mod[0]]['mods'][mod[1]]=0
            aa_occ[mod[0]]['mods'][mod[1]]+=1


for aa in aa_occ:
    for range in ranges:
        if ranges[range]['values'][0] <= float(aa) <= ranges[range]['values'][1]:
            aa_occ[aa]['total']+=ranges[range]['count']

for aa in sorted(aa_occ):
    for mod in aa_occ[aa]['mods']:
        occ=aa_occ[aa]['mods'][mod]/aa_occ[aa]['total']
        print('aa#{} [{}]: Occupancy={:.2f}({}/{})'.format(aa, mod, occ, aa_occ[aa]['mods'][mod], aa_occ[aa]['total']))

aa#100 [Metal:Cu(I) on E]: Occupancy=0.02(5/257)
aa#100 [Metal:Zinc on E]: Occupancy=0.03(7/257)
aa#100 [Metal:Sodium on E]: Occupancy=0.00(1/257)
aa#106 [Metal:Sodium on D]: Occupancy=0.02(4/257)
aa#106 [Metal:Cu(I) on D]: Occupancy=0.06(15/257)
aa#106 [Metal:Zinc on D]: Occupancy=0.07(19/257)
aa#106 [Metal:Calcium on D]: Occupancy=0.05(14/257)
aa#106 [Metal:Potassium on D]: Occupancy=0.05(12/257)
aa#112 [Metal:Sodium on E]: Occupancy=0.13(34/257)
aa#112 [Metal:Calcium on E]: Occupancy=0.04(9/257)
aa#112 [Metal:Potassium on E]: Occupancy=0.03(8/257)
aa#117 [Common Biological:Formylation on K]: Occupancy=0.00(1/257)
aa#150 [UniProt:Phosphoserine on S]: Occupancy=0.65(36/55)
aa#152 [UniProt:Phosphoserine on S]: Occupancy=0.65(36/55)
aa#155 [Common Variable:Oxidation on M]: Occupancy=0.16(9/55)
aa#156 [Common Biological:Phosphorylation on S]: Occupancy=0.05(3/55)
aa#157 [Common Biological:Phosphorylation on T]: Occupancy=0.04(2/55)
aa#161 [Common Artifact:Deamidation on N]: Occupancy=0.0

In [9]:
df2

,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,Delta Score,Notch,Base Sequence,Full Sequence,Essential Sequence,Ambiguity Level,"PSM Count (unambiguous, <0.01 q-value)",Mods,Mods Chemical Formulas,Mods Combined Chemical Formula,Num Variable Mods,Missed Cleavages,Peptide Monoisotopic Mass,Mass Diff (Da),Mass Diff (ppm),Protein Accession,Protein Name,Gene Name,Organism Name,Identified Sequence Variations,Splice Sites,Contaminant,Decoy,Peptide Description,Start and End Residues In Protein,Previous Amino Acid,Next Amino Acid,Theoreticals Searched,Decoy/Contaminant/Target,Matched Ion Series,Matched Ion Mass-To-Charge Ratios,Matched Ion Mass Diff (Da),Matched Ion Mass Diff (Ppm),Matched Ion Intensities,Matched Ion Counts,Normalized Spectral Angle,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue
4,04-30-13_CAST_Frac4_6uL,20504,132.09318,200.0,3.359238e+07,20502,2.0,1049.49248,2096.97040,27.348,22.302,0,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,1,129,NaN,NaN,NaN,0,0,2096.96508,0.00532,2.54,P55095,Glucagon,primary:Gcg,Mus musculus,NaN,NaN,N,N,full,[98 to 117],R,E,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:138.06607, b2+1:209.10255, b3+1:338.1448...","[b1+1:-0.00012, b2+1:-0.00075, b3+1:-0.00103, ...","[b1+1:-0.86, b2+1:-3.59, b3+1:-3.05, b4+1:-3.8...","[b1+1:41940, b2+1:3141376, b3+1:558438, b4+1:2...",27,-1.0,,,5,0.0,0.000000,5,0.0,0.000000,5.722046e-06,0.000002
5,04-30-13_CAST_Frac4_6uL,22044,140.59091,200.0,1.021720e+07,22042,2.0,1049.49240,2096.97024,27.341,22.296,0,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,1,129,NaN,NaN,NaN,0,0,2096.96508,0.00516,2.46,P55095,Glucagon,primary:Gcg,Mus musculus,NaN,NaN,N,N,full,[98 to 117],R,E,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:138.06601, b2+1:209.10260, b3+1:338.1453...","[b1+1:-0.00018, b2+1:-0.00070, b3+1:-0.00051, ...","[b1+1:-1.30, b2+1:-3.37, b3+1:-1.51, b4+1:0.39...","[b1+1:24418, b2+1:1007729, b3+1:141241, b4+1:6...",27,-1.0,,,6,0.0,0.000000,6,0.0,0.000000,2.980232e-07,0.000000
6,04-30-13_CAST_Frac4_6uL,22803,144.79241,200.0,8.736389e+06,22801,2.0,1049.49220,2096.96985,27.333,22.291,0,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,1,129,NaN,NaN,NaN,0,0,2096.96508,0.00477,2.27,P55095,Glucagon,primary:Gcg,Mus musculus,NaN,NaN,N,N,full,[98 to 117],R,E,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:138.06624, b2+1:209.10239, b3+1:338.1442...","[b1+1:0.00005, b2+1:-0.00092, b3+1:-0.00167, b...","[b1+1:0.37, b2+1:-4.40, b3+1:-4.95, b4+1:-6.66...","[b1+1:16234, b2+1:785373, b3+1:130550, b4+1:52...",27,-1.0,,,7,0.0,0.000000,7,0.0,0.000000,2.026558e-06,0.000001
8,04-30-13_CAST_Frac4_6uL,29008,180.49812,200.0,6.538526e+06,29007,2.0,1049.49046,2096.96637,27.328,22.284,0,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,1,129,NaN,NaN,NaN,0,0,2096.96508,0.00129,0.62,P55095,Glucagon,primary:Gcg,Mus musculus,NaN,NaN,N,N,full,[98 to 117],R,E,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:138.06433, b2+1:209.10239, b3+1:338.1449...","[b1+1:-0.00186, b2+1:-0.00092, b3+1:-0.00091, ...","[b1+1:-13.55, b2+1:-4.40, b3+1:-2.69, b4+1:-4....","[b1+1:10692, b2+1:579646, b3+1:93615, b4+1:346...",27,-1.0,,,9,0.0,0.000000,9,0.0,0.000000,1.013279e-05,0.000004
9,04-30-13_CAST_Frac4_6uL,20130,130.05916,200.0,3.098117e+07,20128,2.0,1049.49233,2096.97011,27.328,22.286,0,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,HAEGTFTSDVSSYLEGQAAK,1,129,NaN,NaN,NaN,0,0,2096.96508,0.00504,2.40,P55095,Glucagon,primary:Gcg,Mus musculus,NaN,NaN,N,N,full,[98 to 117],R,E,,T,"[b1+1, b2+1, b3+1, b4+1, b5+1, b6+1, b7+1, b8+...","[b1+1:138.06554, b2+1:209.10255, b3+1:338.1447...","[b1+1:-0.00065, b2+1:-0.00075, b3+1:-0.00118, ...","[b1+1:-4.76, b2+1:-3.59, b3+1:-3.50, b4+1:-5.5...","[b1+1:44845,